In [1]:
import torch
import gensim
import numpy as np

fake data

In [2]:
train_data = torch.randn((100, 125, 300)) #100 sentences of 125 words and 300 is for glove embeding

train_label = torch.ones(100, dtype = torch.int64)

In [3]:
test_data = torch.randn((20, 125, 300)) #20 sentences, each of 125 words and 300 is for glove embeding

train_label = torch.ones(100, dtype = torch.int64)

## LSTM

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class LSTMcell(nn.Module):
    
    def __init__(self, batch_size ,input_size, hidden_size, output_size):
        
        super(LSTMcell, self).__init__()
        
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        """
        LSTM cell basic operations
        """
        self.i2ft = nn.Linear(input_size + hidden_size, hidden_size, bias = True)
        self.i2it = nn.Linear(input_size + hidden_size, hidden_size, bias = True)
        self.i2cdasht = nn.Linear(input_size + hidden_size, hidden_size, bias = True)
        self.i2o = nn.Linear(input_size+hidden_size, hidden_size, bias=True)


    def forward(self, input, hidden_state, cell_state):
        
        """
        input dimension = (batch size X 300); where 300 is dimension used for word embedding
        
        hidden state dimension = (batch size X 300); where 300 is hidden state dimension as mentioned in the paper
        
        """
        
        combined = torch.cat((input, hidden_state), axis = 1)
        
        forget_gate = torch.sigmoid(self.i2ft(combined))
        i_t = torch.sigmoid(self.i2it(combined))
        c_dash = torch.tanh(self.i2cdasht(combined))
        cell_state = forget_gate*cell_state + i_t*c_dash
        
        """
        IMP: Layer normalization [2] to be performed after the computation of the cell state
        """
        output_state = torch.sigmoid(self.i2o(combined))
        hidden_state = output_state*torch.tanh(cell_state)
        
        
        return output_state, hidden_state, cell_state
        
    def initHidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)

In [5]:
class LSTMclassifier(nn.Module):
    
    """
    Classification task on LSTM output
    """
    
    def __init__(self, batch_size ,input_size, hidden_size, output_size, max_num_of_words):
        
        super(LSTMclassifier, self).__init__()
        
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.labels = output_size
        self.max_num_of_words = max_num_of_words
        self.lstm = LSTMcell(batch_size ,input_size ,n_hidden, n_categories)
        
        """
        Pooling layer: mean pooling across time 
                       input dimension: (batch_size X max_num_of_words X 300)
                       output dimension: (batch_size X 300)
        
        """
        self.pool = nn.AvgPool2d((max_num_of_words,1), stride=1)
        self.layer1 = nn.Linear(input_size, hidden_size, bias = True)
        self.layer2 = nn.Linear(hidden_size, output_size, bias = True)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input):
        
        hidden_state = torch.zeros(self.batch_size, self.hidden_size)
        cell_state = torch.zeros(self.batch_size, self.hidden_size)
        
        "output is concatenation of all time stamp"
        output = torch.zeros((self.batch_size, self.max_num_of_words, 300))
        
        for i in range(self.max_num_of_words):
            output_state, hidden_state, cell_state = self.lstm(input[:,i,:], hidden_state, cell_state)
            output[:,i,:] = output_state
        
        time_avg_output = torch.squeeze(self.pool(output))
        linear_layer = torch.sigmoid(self.layer1(time_avg_output))
        final_output = torch.sigmoid(self.layer2(linear_layer))
        final_output = self.softmax(final_output)
        
        return final_output

In [6]:
n_hidden = 300
input_size = 300
n_categories = 5
batch_size = 5

max_num_of_words = 125 # for this case

ann = LSTMclassifier(batch_size ,input_size ,n_hidden, n_categories, max_num_of_words)

In [7]:
criterion = nn.NLLLoss()
learning_rate = 0.005

def train(category_tensor, sentence_vec):
    
    """
    category tensor dimension: (batch_size X num_of_labels)
    sentence_vec dimension: (batch_size X max num of words X 300)
    output dimension: (batch_size X num_of_labels)
    """

    ann.zero_grad()
    
    output = ann(sentence_vec)

    loss = criterion(output, category_tensor)
    loss.backward()

    for p in ann.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [8]:
num_epoch = 2

for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for iter in range(0, train_data.size()[0], batch_size):
        if iter + batch_size < train_data.size()[0]:
            output, loss = train(train_label[iter:(iter+batch_size)], train_data[iter:(iter+batch_size), :, :])
        else:
            output, loss = train(train_label[iter:(train_data.size()[0])], train_data[iter:(train_data.size()[0]), :, :])
#         print("loss:", loss)

        running_loss += loss
    print(running_loss)

print('Finished Training')

28.592833280563354
24.93593692779541
Finished Training
